# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686495


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<00:59,  2.14s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:49,  1.89s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:32,  1.34s/it]

Rendering models:  33%|███▎      | 10/30 [00:07<00:18,  1.05it/s]

...

Rendering models:  67%|██████▋   | 20/30 [00:07<00:03,  2.85it/s]

Rendering models:  77%|███████▋  | 23/30 [00:07<00:01,  3.85it/s]

Rendering models:  87%|████████▋ | 26/30 [00:07<00:00,  5.05it/s]

Rendering models:  97%|█████████▋| 29/30 [00:07<00:00,  6.61it/s]

ElisabethB                            0.000312
not-logged-in-20a275b53abec8db9f41    0.047180
T_atom                                0.001349
not-logged-in-a9247261537fa1da33fe    2.136226
Andy_Carleen                          0.000464
Camelron                              0.000402
shocko61                              0.000213
djswanso                              0.000674
not-logged-in-5a8bb7be433678ade709    0.079893
lsautter                              0.000525
not-logged-in-5f30b2bb9b7f9d49da41    0.004175
orzechowskijacek                      0.000605
acapirala                             0.000409
arad98                                0.000438
Lavadude                              0.192010
clars915                              0.001436
jordanrushworth                       0.003585
Tessaje04                             0.007253
LeusaneLordelo                        0.001108
FrezeON                               0.000484
ykuzmits                              0.000345
mhaloftis    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())